In [18]:
import cv2
import numpy as np

In [19]:
cam = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")

face_data = []
data_path = "./Datasets/"
skip= 0

file_name = input("Enter the name of the face : ")
while True:
    ret_value, frame = cam.read()
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    if ret_value == False:
        continue

    faces = face_cascade.detectMultiScale(gray_frame,1.3,5)
    faces = sorted(faces, key=lambda f:f[2]*f[3])

    for (x,y,w,h) in faces[-1:]:
        cv2.rectangle(gray_frame, (x,y), (x+w, y+h), (255,255,0),2)

        offset = 10
        face_selection = gray_frame[y-offset:y+h+offset, x-offset:x+w+offset] #the whole face should be covered and thus we add the offset at the end to include extra area as a buffer
        face_selection = cv2.resize(face_selection,(100,100))
        skip += 1
        if skip%10 == 0:
            face_data.append(face_selection)
            print(len(face_data))

    cv2.imshow("Video Frame", gray_frame)
    cv2.imshow("face Selection",face_selection)
    
    key_pressed = cv2.waitKey(1) & 0xFF
    if key_pressed == ord('s'):
        break

face_data = np.asarray(face_data)
face_data = face_data.reshape((face_data.shape[0],-1))

np.save(data_path+file_name+'.npy', face_data)

cam.release()
cv2.destroyAllWindows()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26


Extracting faces

In [20]:
import os

knn algo

In [21]:
def distance(x1, x2):
    return np.sqrt(((x1-x2)**2).sum())

def knn(train, test_point, k=5):
    distances = []

    for i in range(train.shape[0]):
        ix = train[i, : -1]
        iy = train[i, -1]

        d = distance(test_point, ix)
        distance.append([d, iy])

    dk = sorted(distances, key=lambda x:x[0] )[:k]
    labels = np.array(dk)[:, -1]


    output = np.unique(labels, return_counts=True)
    index = np.argmax(output[1])
    return output[0][index]

In [22]:
def knn(train, test, k=5):
    distances = []

    for i in range(train.shape[0]):
        ix = train[i, :-1]
        iy = train[i, -1]

        d = distance(test, ix)
        distances.append([d, iy])
    distance_k = sorted(distances, key=lambda x:x[0])
    distance_k = distance_k[:k]

    labels = np.array(distance_k)[:,-1]

    output = np.unique(labels, return_counts=True)
    index_most_frequnt = np.argmax(output[1])
    return output[0][index_most_frequnt]


In [23]:
cam = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")

face_data = []
data_path = "./Datasets/"
skip= 0

labels = []

class_id = 0
names = {}

In [24]:
for files in os.listdir(data_path):
    if files.endswith(".npy"):
        names[class_id] = files[:-4]
        print("Loaded = "+files)
        data_file = np.load(data_path+files)
        face_data.append(data_file)

        target = class_id*np.ones((data_file.shape[0],))
        class_id += 1
        labels.append(target)

face_dataset = np.concatenate(face_data, axis=0)
face_labels = np.concatenate(labels, axis=0).reshape((-1,1))

print(face_dataset.shape)
print(face_labels.shape)

trainset = np.concatenate((face_dataset, face_labels), axis=1)
print(trainset.shape)

Loaded = Sanidhyagoel.npy
(26, 10000)
(26, 1)
(26, 10001)


In [25]:
while True:
    ret_value, frame = cam.read()
    if ret_value == False:
        continue

    faces = face_cascade.detectMultiScale(frame,1.3,5)

    for (x,y,w,h) in faces:
        offset = 10
        face_selection = frame[y-offset:y+w+offset, x-offset:x+h+offset]
        face_selection = cv2.resize(face_selection, (100,100))

        out = knn(trainset, face_selection.flatten())

        pred_name = names[int(out)]
        cv2.putText(frame, pred_name, (x,y-10), cv2.FONT_HERSHEY_COMPLEX, 1,(255,0,0),2,cv2.LINE_AA)
        cv2.rectangle(frame, (x,y), (x+h, y+w), (0,255,255), 2)    
    cv2.imshow("faces", frame)

    key_pressed = cv2.waitKey(1) & 0xFF
    if key_pressed == ord("s"):
        break

cam.release()
cv2.destroyAllWindows()

error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\demosaicing.cpp:1699: error: (-215:Assertion failed) scn == 1 && dcn == 1 in function 'cv::demosaicing'
